You can use this template for problems 1, 2 and 3

In [1]:
import numpy as np
import time
# Students will submit their files with their team-name.py
# Student have to use the Team as their parent class

Recent Version of the Code

In [120]:
class AE20B108_Q2():
  def __init__(self):
    self.__num_actions = 6
    self.__sample_means = np.zeros(self.__num_actions)
    self.__arm_pulls = np.zeros(self.__num_actions)

  @staticmethod
  def KL(x, y, epsilon = 1e-15):
    x = min((max(x, epsilon)), 1-epsilon)
    y = min((max(y, epsilon)), 1-epsilon)
    return x * np.log(x/y) + (1 - x) * np.log((1 - x)/(1 - y))

  @staticmethod
  def KLmaxUCB(c, t):
    return np.log(t) + c * np.log(np.log(t))

  @staticmethod
  def KLUpperConfidenceBound(p, u, c, t):
    maxBound = AE20B108_Q2.KLmaxUCB(c, t) / u
    q = p
    b = (1 - p)/2
    while b > 1e-5:
      if AE20B108_Q2.KL(p, q + b) <= maxBound:
        q += b
      b /= 2
    return q

  def get_action(self, wicket, runs_scored):
    t = int(np.sum(self.__arm_pulls)) + 1
    if t <= self.__num_actions:
      action = t - 1
      self.__arm_pulls[action] += 1
      self.__sample_means[action] += (runs_scored)
    else:
      KL_estimates = [self.KLUpperConfidenceBound(p = self.__sample_means[i]/self.__arm_pulls[i], u = self.__arm_pulls[i], c = 3, t = t) for i in range(self.__num_actions)]
      action = np.argmax(KL_estimates)
      self.__arm_pulls[action] += 1
      self.__sample_means[action] += runs_scored
    return action


In [121]:
class Environment:
  def __init__(self,num_balls,agent):
    self.num_balls = num_balls
    self.agent = agent
    self.__run_time = 0
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0
    self.__start_time = 0
    self.__end_time = 0
    self.__regret_w = 0
    self.__regret_s = 0
    self.__wicket = 0
    self.__regret_rho = 0
    self.__p_out = np.array([0.001,0.01,0.02,0.03,0.1,0.3])
    self.__p_run = np.array([1,0.9,0.85,0.8,0.75,0.7])
    self.__action_runs_map = np.array([0,1,2,3,4,6])
    self.__s = (1-self.__p_out)*self.__p_run*self.__action_runs_map
    self.__rho = self.__s/self.__p_out


  def __get_action(self):
    self.__start_time      = time. time()
    action          = self.agent.get_action(self.__wicket,self.__runs_scored)
    self.__end_time        = time. time()
    self.__run_time   = self.__run_time + self.__end_time - self.__start_time
    return action


  def __get_outcome(self, action):
    pout = self.__p_out[action]
    prun= self.__p_run[action]
    wicket = np.random.choice(2,1,p=[1-pout,pout])[0]
    runs = 0
    if(wicket==0):
      runs = self.__action_runs_map[action]*np.random.choice(2,1,p=[1-prun,prun])[0]
    return wicket, runs


  def innings(self):
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0

    for ball in range(self.num_balls):
      action = self.__get_action()
      self.__wicket, self.__runs_scored   = self.__get_outcome(action)
      self.__total_runs     = self.__total_runs + self.__runs_scored
      self.__total_wickets  = self.__total_wickets + self.__wicket
      self.__regret_w       = self.__regret_w+ (self.__p_out[action]-np.min(self.__p_out))
      self.__regret_s       = self.__regret_s+ (np.max(self.__s) - self.__s[action])
      self.__regret_rho       = self.__regret_rho+ (np.max(self.__rho)-self.__rho[action])
    return self.__regret_w,self.__regret_s,self.__regret_rho, self.__total_runs, self.__total_wickets, self.__run_time


In [134]:
agent = AE20B108_Q2()
environment = Environment(100,agent)
regret_w,regret_s,reger_rho,total_runs,total_wickets,run_time = environment.innings()
print(regret_w,regret_s,reger_rho,total_runs,total_wickets,run_time)

28.56099999999997 7.114999999999997 7702.000000000012 322 28 0.030465126037597656
